In [1]:
%load_ext Cython

In [2]:
%%cython -a --cplus
#cython: cdivision = True
##cython: profile=True


from libc.stdlib cimport malloc,free
from libcpp.vector cimport vector
from libcpp.string cimport string

from numpy.math cimport INFINITY
import time
import sys
    
cdef float INF = INFINITY
ctypedef unsigned long long int64

cdef struct move:
    int start
    int end
    int die
    
cdef struct move_group:
    int start[4]
    int end[4]
    int n
    
cdef move get_move(int a, int b):
    cdef move m
    m.start = a
    m.end = b
    return m

    
cdef struct log_entry:
    string action #
    move m
    int dice[2]# if roll, contains the dice results, if a move contains start and end locations, if double means nothing
    
    
#we will define some constant arrays purely for clarity of code. These arrays encode the location of the bar and the home
cdef int *BAR = [25,0]
cdef int *BEAROFF = [0,25]
   

cdef class Position(object):
    cdef:
        int board[26]
        int cube,n_dice,player,value
        int dice[4]
        int ilog
#         vector[log_entry] log
        log_entry log[5000]
        int log_size
        int borne[2]
            
    def __cinit__(self):
        self.board = [0,-2,0,0,0,0,5,0,3,0,0,0,-5,5,0,0,0,-3,0,-5,0,0,0,0,2,0] #not sure if 1 array or 2 arrays is better
        self.player = 1
        self.value = -1
        self.cube = 1
        self.n_dice = 0 
        self.dice = [0,0,0,0]
        self.log_size = 0
        self.borne = [0,0]
        self.ilog = 0
#         self.log.reserve(5000)
        
    cdef void play(self,move m):
        cdef log_entry entry
        #checks if there are pieces to move
        assert (self.n_dice > 0), "no moves left"
        assert (self.board[m.start]*self.value > 0),  "no piece to move"+str(m)
            
        #checks if location has at most 1 enemy piece
        assert (m.end == BEAROFF[self.player] or self.board[m.end]*self.value >= -1) , "destination is blocked"
        entry.m = m
        self.board[m.start] -= self.value
        entry.action = 'ordinary'
        
        if m.end == BEAROFF[self.player]:
            self.borne[self.player] += 1
            entry.action = 'bearoff'
        else:
            if self.board[m.end] * self.value == -1: #capture
                self.board[m.end] = 0
                self.board[BAR[1^self.player]] -=self.value
                entry.action = 'capture'
            self.board[m.end] += self.value
        
        entry.dice = [0,0]
        self.use_die(m.die)
#         self.log.push_back(entry)
#         self.log_size += 1  
        self.log[self.ilog] = entry
        self.ilog += 1
#         if not self.n_dice:
#             self.player ^= 1
#             self.value *= -1
        
            
    cdef void use_die(self,int die):
        cdef int i,j
        i = 0
        while die < self.dice[i] and i < self.n_dice-1:
            i += 1
        assert(0<=i<4),'i is {}, die is {}'.format(i,die)
        assert(die == self.dice[i]),'die is {}, i is {}, self.dice[i] is {}'.format(die,i,self.dice[i])
        self.n_dice -= 1
        for j in xrange(i, self.n_dice):
            assert(0<=j<3),'j is {}, die is {}'.format(j,die)
            self.dice[j] = self.dice[j+1]
        
        
    
    cdef void put_die(self,int die):
        cdef int i,j
        assert(die in range(1,7))
        assert(self.n_dice < 4)
        i = 0
        while die < self.dice[i] and i < self.n_dice:
            i += 1
        assert(0<=i < 4)
        self.n_dice += 1
        for j in xrange(self.n_dice-1, i , -1):
            assert(1<=j<=3)
            self.dice[j] = self.dice[j-1]
        self.dice[i] = die
        
            
            
        
    cdef void roll(self):
        cdef log_entry entry
        cdef int swap
        
        self.player ^= 1
        self.value *= -1        
        self.dice = [0,0,0,0]
        self.n_dice = 2
        roll(self.dice,2)
        if self.dice[0]==self.dice[1]:
            self.dice[2]=self.dice[3]=self.dice[0]
            self.n_dice = 4

        #after storing into the log, we sort the dice from largest to smallest
        
        if self.n_dice == 2 and self.dice[0] < self.dice[1]:
            swap = self.dice[0]
            self.dice[0] = self.dice[1]
            self.dice[1] = swap
            
        entry.action = 'roll'
        entry.dice[0] = self.dice[0]
        entry.dice[1] = self.dice[1]
        entry.m.start = 0
        entry.m.end = 0
        entry.m.die = 0
#         self.log.push_back(entry)
        self.log[self.ilog] = entry
        self.ilog += 1
        
    cdef void force_roll(self,int die1,int die2):
        cdef log_entry entry
        cdef int swap
        self.player ^= 1
        self.value *= -1     
        self.dice = [die1,die2,die2,die2]
        self.n_dice = 2 if die1 != die2 else 4
        entry.action = 'roll'
        if self.n_dice == 2 and self.dice[0] < self.dice[1]:
            swap = self.dice[0]
            self.dice[0] = self.dice[1]
            self.dice[1] = swap    
        entry.dice[0] = die1
        entry.dice[1] = die2
        entry.m.start = 0
        entry.m.end = 0
        entry.m.die = 0
#         self.log.push_back(entry)
        self.log[self.ilog] = entry
        self.ilog += 1
        
    cdef void switch(self):
#         self.player ^= 1
#         self.value *= -1
        
        cdef log_entry e        
        e.action = "switch"
        e.dice = [self.dice[0],self.dice[1]]
        e.m.start = 0
        e.m.end = 0
        e.m.die = self.n_dice
        self.n_dice = 0
#         self.log.push_back(e)
        self.log[self.ilog] = e
        self.ilog += 1
            
        
    cdef back(self):
        cdef log_entry action
        
#         print self.log[-1:]
#         print 'size' , self.log.size()     
#         raw_input('fetching back')
#         print self.log.back()
#         print self.log[-1:]
#         action = self.log.back()
#         action = self.log.at(self.log.size()-1)
#         action = self.log[-1:][0]
#         raw_input('got back')

        self.ilog -= 1
        action = self.log[self.ilog]
        
#         self.log.pop_back()
        if action.action == 'ordinary' or  action.action == 'capture' or  action.action == 'bearoff':
            assert(action.m.start in range(26) and action.m.end in range(26) and action.m.die in range(1,7),
                   'start end, or die out of range')
#             if not self.n_dice:
#                 self.player ^= 1
#                 self.value *= -1
            self.board[action.m.start] += self.value
            if action.action == 'capture':
                self.board[action.m.end] = -self.value
                assert(self.board[BAR[1^self.player]] , 'nothing on bar')
                self.board[BAR[1^self.player]] += self.value
            elif action.action == 'bearoff':  
                self.borne[self.player] -= 1
            else:
                self.board[action.m.end] -= self.value
            self.put_die(action.m.die)
   
        if action.action == 'roll':
            self.n_dice = 0
            self.player ^= 1
            self.value *= -1
            
        if action.action == 'switch':
            self.n_dice = action.m.die
            self.dice[0] = action.dice[0]
            self.dice[1] = action.dice[1]
            if self.n_dice>2:
                self.dice[2] = self.dice[0]
            if self.n_dice == 4:
                self.dice[3] = self.dice[0]            
#             self.player ^= 1
#             self.value *= -1
            None
            
    cdef int can_bearoff(self):
        cdef int count
        cdef int i
        count = self.borne[self.player]
        for i in xrange(1,7):
            if self.board[BEAROFF[self.player] + self.value * i] * self.value > 0:
                count += self.board[BEAROFF[self.player] + self.value * i] * self.value
        return count == 15
        
            
    cdef int can_play(self , int die):
        #the cases for both players are so different, we can consider two different functions
        # find out if a die can be played
        cdef int i
        cdef int overflow
        # check if there are checkers on the bar
        
        if self.board[BAR[self.player]]:
            return self.board[BAR[self.player] - self.value * die] * self.value >= -1
        
        #check if there is an ordinary or capture move
        for i in xrange(BEAROFF[0] + 1 + die*(1^self.player) , BEAROFF[1] - die * self.player):
            if self.board[i] * self.value > 0 and self.board[i - self.value * die] * self.value >= -1:
                return 1
        
        #if we are bearing off, check if we can bear off
        
        if not self.can_bearoff(): #we can't bear off
            return 0
        i = 6
        while i > die:
            if self.board[BEAROFF[self.player] + self.value * i] * self.value > 0:
                return 0 #no bearoff
            i -= 1
        return 1 #we can bearoff
    
    cdef int can_move(self , move m):
        if m.end<0 or m.end > 25:
            return 0
        #is there a checker on the bar?
        if self.board[BAR[self.player]]:
            return m.start == BAR[self.player] and self.board[m.end] * self.value >= -1        
        # try a regular move
        if BEAROFF[0] < m.end < BEAROFF[1]:
            return self.board[m.start] * self.value > 0 and self.board[m.end] * self.value >= -1
        #try bearing off
        cdef int overflow
        for overflow in xrange(BEAROFF[self.player],BEAROFF[self.player] + 7*self.value,self.value):
            if self.board[i]*self.value > 0:
                break
        return (m.end == BEAROFF[self.player] and self.can_bearoff() and self.board[m.start] * self.value > 0
                 and (abs(m.start-m.end) == m.die or (m.start==overflow and m.die > abs(m.end-m.start))))
    
    
    
    #A function that will determine if a roll can be played arbitrarily this turn 
    #Helpful for generating moves
    
    cdef int can_play_more_than_one(self , int die, int* exception):
        cdef int playable = 0
        cdef move m
        cdef int i
        m.die = die
        exception[0] = -1 #in the case that there is only one playable move, we record where it is
        #if there is a checker on the board, check for one playable move
        if self.board[BAR[self.player]] == self.value:
            playable = 1 #artificially increase playable to one because we only need to play 1 move
        if self.board[BAR[self.player]]*self.value > 1:
            return 2
        for m.start in xrange(BEAROFF[0] + 1 + (die)*(1^self.player) , BEAROFF[1] - (die) * self.player):
            m.end = m.start - die*self.value
            if self.can_move(m):
                exception[0] = m.start
                playable += self.board[m.start]*self.value
                if playable >1:
                    return playable                
        #bearing off
        if not self.can_bearoff():
            return playable                
        i = 6
        overflow = 1
        while i > die:
            if self.board[BEAROFF[self.player] + self.value * i] * self.value > 0:
                overflow = 0 #no bearoff
                break
            i -= 1        
        m.end = BEAROFF[self.player]
        m.start = m.end + die*self.value
        if self.can_move(m):
            overflow = 0
            playable += self.board[m.start] * self.value
            exception[0] = m.start
            if playable > 1:
                return playable            
        i = die - 1
        while overflow and i > 0:
            m.start = m.end + i*self.value
            if self.can_move(m):
                overflow = 0
                playable += self.board[m.start] * self.value
                exception[0] = m.start
                if playable > 1:
                    return playable
            i -= 1            
        return playable
    
      
            
    cdef int get_moves(self, vector[move]& moves): #return if there are legal moves
        ''' this function checks on the fly if more than one move can be played'''
        cdef int max_moves = 0
        cdef int i,die,overflow
        cdef int count = 0
        cdef move m
        cdef int two_values #whether there are two dice values or one
        cdef int play_after[2]
        cdef int exception0[1],exception1[1]
        cdef int accept_all
        
        moves.clear()
        
        if self.n_dice == 0:
            return 0
        
        two_values = self.dice[0] != self.dice[1] and self.n_dice == 2
        
        if two_values:
            play_after[0] = self.can_play_more_than_one(self.dice[0],exception0)
            play_after[1] = self.can_play_more_than_one(self.dice[1],exception1)
        else:
            play_after[0] = self.can_play(self.dice[0])
            play_after[1] = 0
            
        if not (play_after[0] or play_after[1]):
            return 0
#         print 'two values' , two_values
#         print 'play after larger',play_after[0]
#         print 'play after smaller',play_after[1]
        
        #check the bar for checkers
        if self.board[BAR[self.player]]:
            max_moves = 0
            #larger die can be played
            m.start = BAR[self.player]
            m.end = BAR[self.player] - self.dice[0] * self.value
            m.die = self.dice[0]
            if self.can_move(m):
                moves.push_back(m)
                max_moves = 1
                if not two_values:
                    return 1
                if play_after[1]:
                    max_moves = 2
                else:
                    self.play(m)
                    if self.can_play(self.dice[1]):
                        max_moves = 2
                    self.back()
            #smaller die can be played  
            m.end = BAR[self.player] - self.dice[1] * self.value
            m.die = self.dice[1]
            if two_values and self.can_move(m):
                if max_moves == 0:
                    moves.push_back(m)
                    return 1
                
                if play_after[0]:                    
                    if max_moves == 1:
                        moves.clear()
                    moves.push_back(m)
                    max_moves = 2
                else:
                    self.play(m)
                    if self.can_play(self.dice[0]):
                        if max_moves == 1:
                            moves.clear()
                        moves.push_back(m)
                        max_moves = 2
                    self.back()
                return max_moves
            return max_moves
        
        #a short check to see if any valid move is accepted. Don't know if it is even worth it to include this
        accept_all = 0
        if (two_values and ((play_after[0]>1 and play_after[1]) or (play_after[0] and play_after[1] > 1) or
                (play_after[0] and play_after[0] and exception0[0] != exception1[0]))) or (not two_values and play_after[0]):
            accept_all = 1
            max_moves = 2                
            
#         print "accepting any move",accept_all
        #normal move or capture
        if play_after[0]:
            die = self.dice[0]
            m.die = die
            if max_moves ==0:
                max_moves = 1            
#             for m.start in xrange(BEAROFF[0] + 1 + (die)*(1^self.player) , BEAROFF[1] - (die) * self.player):
            for m.start in xrange(1,25):
                m.end = m.start - die*self.value
                if self.can_move(m):
                    if not two_values:
                        moves.push_back(m)
                        continue
                    if accept_all or play_after[1]>1 or (play_after[1] and exception1[0]):
                        if max_moves == 1:
                            moves.clear()
                        moves.push_back(m)
                        max_moves = 2
                    else:
                        self.play(m)
                        if self.can_play(self.dice[1]):
                            if max_moves == 1:
                                moves.clear()
                            max_moves = 2
                            moves.push_back(m)
                        else:
                            if max_moves == 1:
                                moves.push_back(m)                                 
                        self.back()
                        
           
            #check for overflow bearoffs
            overflow = 1
            m.end = BEAROFF[self.player]
            i = 6
            while overflow and i>0:
                m.start = m.end + i*self.value
                if self.can_move(m):
                    overflow = 0
                    if i < die:
                        if not two_values:
                            moves.push_back(m)
                            continue
                        if max_moves == 0:
                            max_moves = 1
                        if accept_all or play_after[1]>1 or (play_after[1] and exception1[0]):
                            moves.push_back(m)
                            max_moves = 2
                        else:
                            self.play(m)
                            if self.can_play(self.dice[1]):
                                if max_moves == 1:
                                    moves.clear()
                                    max_moves = 2
                                moves.push_back(m)
                            elif max_moves == 1:
                                moves.push_back(m) 
                            self.back()
                i -= 1
        
                            
        if not two_values:
            return max_moves
        
        #normal move or capture
        if play_after[1]:
            die = self.dice[1]
            m.die = die
            if max_moves == 0:
                max_moves = 1
#             for m.start in xrange(BEAROFF[0] + 1 + die*(1^self.player) , BEAROFF[1] - die * self.player):
            for m.start in xrange(1,25):
                m.end = m.start - die*self.value
                if self.can_move(m):
                    if accept_all or play_after[0]>1 or (play_after[0] and exception0[0]):
                        if max_moves == 1:
                            moves.clear()
                        max_moves =2
                        moves.push_back(m)
                    else:
                        self.play(m)
                        if self.can_play(self.dice[1]):
                            if max_moves == 1:
                                moves.clear()
                            max_moves = 2
                            moves.push_back(m)
                        else:
                            if max_moves == 1 and not play_after[0]:
                                moves.push_back(m)                                 
                        self.back()
                        
           
            #check for overflow bearoffs
            overflow = 1
            m.end = BEAROFF[self.player]
            i = 6
            while overflow and i>0:
                m.start = m.end + i*self.value
                if self.can_move(m):
                    overflow = 0
                    if i < die:
                        if max_moves == 0:
                            max_moves = 1
                        if accept_all or play_after[1]>1 or (play_after[1] and exception1[0]):
                            if max_moves == 1:
                                moves.clear()
                            max_moves = 2
                            moves.push_back(m)
                        else:
                            self.play(m)
                            if self.can_play(self.dice[0]):
                                if max_moves == 1:
                                    moves.clear()
                                max_moves = 2
                                moves.push_back(m)
                            elif max_moves == 1 and not play_after[0]:
                                moves.push_back(m) 
                            self.back()
                i -= 1
        return max_moves
            
        
    cdef int move_group(self , vector[vector[move]]& move_groups): #returns how many moves can be made
        assert(self.n_dice == 2 or self.n_dice == 4)
        cdef vector[move] move_group
        cdef move m[4]
        cdef int bearoff_count
        cdef int i,j,k,l,die_index,playable,die
        move_groups.clear()
            
        if self.n_dice == 2:
            ##########checker on bar##########
            for i in xrange(26):
                m[0].start = i
                for die_index in xrange(2):
                    m[0].die = self.dice[die_index]
                    m[0].end = m[0].start - m[0].die*self.value
                    if m[0].end < 0:
                        m[0].end = 0
                    if m[0].end > 25:
                        m[0].end = 25
                    if self.can_move(m[0]):
                        self.play(m[0])
                        for j in xrange(26):
                            m[1].start = j
                            m[1].die = self.dice[1^die_index]
                            m[1].end = m[1].start - m[1].die*self.value
                            if m[1].end < 0:
                                m[1].end = 0
                            if m[1].end > 25:
                                m[1].end = 25
                            if self.can_move(m[1]):
                                if (m[0].die<m[1].die and m[0].end == m[1].start and m[1].end != BEAROFF[self.player] and
                                    self.board[m[0].start-m[1].die*self.value]*self.value > -2): #avoid redundant moves
                                    #todo: find a nice way to eliminate bearoff redundant moves
                                    continue
                                move_group.clear()
                                move_group.push_back(m[0])
                                move_group.push_back(m[1])
                                move_groups.push_back(move_group)
                        self.back()
            if move_groups.size():
                # trim equivalent moves
                return 2
            else:
                for i in xrange(2):
                    m[0].die = self.dice[0]
                    for j in xrange(26):
                        m[0].start = j
                        m[0].end = m[0].start - m[0].die*self.value
                        if m[0].end<0:
                            m[0].end = 0
                        if m[0].end>25:
                            m[0].end = 25
                        if self.can_move(m[0]):
                            move_group.clear()
                            move_group.push_back(m[0])
                            move_groups.push_back(move_group)
                    if move_groups.size():
                        break
                if move_groups.size():
                    return 1
                else:
                    return 0
        else:# 4 dice
            playable = 0
            die = self.dice[0]
            m[0].die=m[1].die=m[2].die=m[3].die=die
            for i in xrange(26):
                m[0].start = i
                m[0].end = i-die*self.value
                m[0].end = min(26,max(0,m[0].end))
                if self.can_move(m[0]):
                    if playable < 1:
                        playable = 1
                        move_groups.clear()
                    if playable == 1:
                        move_group.clear()
                        move_group.push_back(m[0])
                        move_groups.push_back(move_group)
                    playable = max(playable,1)
                    self.play(m[0])
                    for j in xrange(i*self.player,26+(i-25)*(1^self.player)):
                        m[1].start = j
                        m[1].end = j-die*self.value
                        m[1].end = min(26,max(0,m[1].end))
                        if self.can_move(m[1]):
                            if playable < 2:
                                playable = 2
                                move_groups.clear()
                            if playable == 2:
                                move_group.clear()
                                move_group.push_back(m[0])
                                move_group.push_back(m[1])
                                move_groups.push_back(move_group)
                            self.play(m[1])
                            for k in xrange(j*self.player,26+(j-25)*(1^self.player)):
                                m[2].start = k
                                m[2].end = k-die*self.value
                                m[2].end = min(26,max(0,m[2].end))
                                if self.can_move(m[2]):
                                    if playable < 3:
                                        playable = 3
                                        move_groups.clear()
                                    if playable == 3:
                                        move_group.clear()
                                        move_group.push_back(m[0])
                                        move_group.push_back(m[1])
                                        move_group.push_back(m[2])
                                        move_groups.push_back(move_group)
                                    self.play(m[2])
                                    for l in xrange(k*self.player,26+(k-25)*(1^self.player)):
                                        m[3].start = l
                                        m[3].end = l-die*self.value
                                        m[3].end = min(26,max(0,m[3].end))
                                        if self.can_move(m[3]):
                                            if playable < 4:
                                                playable = 4
                                                move_groups.clear()
                                            if playable == 4:
                                                move_group.clear()
                                                move_group.push_back(m[0])
                                                move_group.push_back(m[1])
                                                move_group.push_back(m[2])
                                                move_group.push_back(m[3])
                                                move_groups.push_back(move_group)
                                    self.back()
                            self.back()
                    self.back()
            return playable
            
    cdef string print_move_group(self , vector[vector[move]]& move_groups):
        cdef int i,j
        cdef string s
        s = <string>''
        for i in xrange(move_groups.size()):            
            for j in xrange(move_groups[i].size()):
                s.append(<string>'{:2d}->{:2d}'.format(move_groups[i][j].start,move_groups[i][j].end))
                s.append(<string>' ')
            s.append(<string>'\n')
        return s
        
        
    cdef int check_win(self): #also returns the result
        cdef int count
        
        if self.borne[0]==15:            
            if self.borne[1]:
                return 1 # not gammon
            for i in xrange(BAR[1],BAR[1]+7):#p0 home and bar
                if self.board[i] < 0:
                    return 3 #backgammon
            return 2 #gammon
        if self.borne[1] == 15:      
            if self.borne[0]:
                return -1 # not gammon
            for i in xrange(BAR[0]-6,BAR[0] + 1):#p1 home and bar
                if self.board[i] > 0:
                    return -3 #backgammon
            return -2 #gammon
        
        return 0
    
        
    
    
    

            
    cpdef print_vitals(self):
        print 'board', self.board
        print 'dice left' , [self.dice[i] for i in xrange(self.n_dice)]
        print 'number of dice left', self.n_dice
        print 'player' , self.player
#         print 'log length' , self.log.size()
        print 'log length' , self.ilog
    
    
        print 'checkers borne off' , self.borne
    def ascii_dice(self,n):
        row1 = ' ----- '
        row2 = '|'
        row2 += ' ' if n == 1 else 'o'
        row2 += '   '
        row2 += ' ' if n<4 else 'o'
        row2 += '|'
        row3 = '|'
        row3 += '  o  ' if n%2 else 'o   o' if n==6 else '     '
        row3 += '|'
        row4 = row2[::-1]
        row5 = row1
        return [row1,row2,row3,row4,row5]
        
    cpdef print_board(self):
        s = ''
        for i in xrange(12,-1,-1):
            s += '{:2}'.format(str(i))
            if self.board[i] > 0:
                s += '\033[1;30;'
            else:
                s += '\033[1;31;'

            if i ==0 :
                s+= '43m'
            elif i%2:
                s += '47m'
            else:
                s += '41m'
                
            if self.board[i] > 0:
                s += 'o'*self.board[i] + ' '*(15-self.board[i])
            else:
                s += 'o'*(-self.board[i]) + ' '*(15 + self.board[i])
            
            if i == 0:
                s += '\033[1;30;43m '
            else:
                s += '\033[1;30;40m '
                
            j = 25-i
            
            if self.board[j] > 0:
                s += '\033[1;30;'
            else:
                s += '\033[1;31;'
            if j ==25 :
                s+= '43m'
            elif j%2:
                s += '47m'
            else:
                s += '41m'
                
            if self.board[j] > 0:
                s += ' '*(15-self.board[j]) + 'o'*self.board[j]
            else:
                s += ' '*(15 + self.board[j]) + 'o'*(-self.board[j])
                
            s += '\033[1;30;49m' + '{:2}'.format(str(j))+'\n'
            
        s+= '\033[1;30;49m'
        ### lets print some dice
        rows = ['','','','','']
        
        for i in xrange(self.n_dice):
            die = self.ascii_dice(self.dice[i])
            for j in xrange(5):
                rows[j] += die[j] + ' '
                
        for j in xrange(5):
            s += rows[j]+'\n'
                      
                               
        
        return s
    

        
    cdef play_random(self):
        cdef vector[move] moves
        cdef int64 r
        cdef int size
        self.get_moves(moves)
        size = moves.size()
        if size:
            r = rng64()
            self.play(moves[r%size])
        else:
            self.switch()
            
        
        
######################## Python Wrappers ########################
                        
            
    cpdef p_play(self,int start,int end,int die = -1):
        cdef move m
        if die == -1:
            m.die = abs(end-start)
        else:
            m.die = die
        m.start = start
        m.end=end
        self.play(m)
    cpdef p_back(self):
        self.back()
        
    cpdef p_switch(self):
        self.switch()
        
    cpdef p_roll(self):
        self.roll()
        
    cpdef p_force_roll(self,die1,die2):
        self.force_roll(die1,die2)
        
    cpdef p_get_moves(self):
        cdef vector[move] moves
        self.get_moves(moves)
        
        return ["{}-{}".format(m.start,m.end) for m in moves]
        
    cpdef p_play_first(self):
        cdef vector[move] moves
        self.get_moves(moves)
        cdef move m
        m = moves[0]
        self.play(m)
        
    cpdef p_play_random(self):
        self.play_random()
        
    cpdef p_check_win(self):
        return self.check_win()
    
    cpdef p_get_n_dice(self):
        return self.n_dice
    
    cpdef p_log(self):
        return self.log
    
    cpdef p_ilog(self):
        return self.ilog
    
    cpdef p_can_bearoff(self):
        return self.can_bearoff()
    
    cpdef p_set_board(self,board):
        self.board = board
        self.borne[0] = 15 - sum([x for x in board if x>0])
        self.borne[1] = 15 + sum([x for x in board if x<0])
        
    cpdef p_get_move_group(self):
        cdef vector[vector[move]] groups
        cdef string s
        self.move_group(groups)
        s = self.print_move_group(groups)
        return s
        
        
'''******************Monte Carlo Tree Search******************'''
        
cdef int64 seed[2]

seed[1] = 42

cdef roll(int* rolls, int n = 2):
    cdef int i
    for i in xrange(n):
        rolls[i] = <int> (rng64()%6+1)#tiny amount of bias, I don't care  
    
cdef int64 rng64():
    cdef int64 x,y
    x=seed[0]
    y=seed[1]
    seed[0]=y
    x^=x<<23
    seed[1]=x^y^(x>>17)^(y>>26)
    return seed[1]+y



cdef cppclass Node:
    vector[double] N
    vector[double] V
    vector[double] Q
    vector[Node*]  children
    string role
    Node* parent
    vector[move] moves
    int index
    int leaf
    int root

#silly constructor for Node
cdef Node make_Node():
    cdef Node a
    return a

cdef int select(Node* n , int puct_constant = 1):
    cdef int i
    cdef double q
    cdef double u
    cdef double sqrt_sum = 0.
    cdef int role
    cdef int index
    cdef int r
    cdef Node* child
    cdef double best
    
    if n[0].role == <string>'chance':
        r = rng64()%36
        i = -1
        while r >= 0:
            i += 1     
            r -= PROBS[i]
        return i
        
    elif n[0].role == <string>'max' or n[0].role == <string>'min':
        role = 1 if n[0].role == <string>'max' else -1    
        for i in xrange(n[0].N.size()):
            sqrt_sum += n[0].N[i]
        sqrt_sum = sqrt_sum**0.5
        best = -INF
        index = 0

        for i in xrange(n[0].moves.size()): #puct selection algorithm based on alphago MCTS
            q = role * n[0].Q[i] #exploitation
            u = puct_constant * sqrt_sum / (1 + n[0].N[i]) #exploration
            if q+u > best:
                best = q+u
                index = i
        return index
    elif n[0].role == <string>'switch':
        return 0

#updates the tree with the result (does not modify n)
cdef backprop(Node* n , double result):
    cdef Node* r    
    r = n
    while not r[0].root:
        index = r[0].index
        r = r.parent
        r[0].N[index] += 1
        r[0].V[index] += result
        r[0].Q[index] = r[0].V[index] / r[0].N[index]
        

cdef int i,j
cdef vector[move] ROLLS
ROLLS = [get_move(i,j) for i in xrange(1,7) for j in xrange(i,7)]
cdef int PROBS[21]
PROBS = [1 if i==j else 2 for i in xrange(1,7) for j in xrange(i,7)]

cdef expand(Node* n,Position p):
    cdef int size
    cdef Node* child
    cdef int i,j
    cdef move dummy_move 
    cdef vector[move] test_moves
    
    if p.check_win():
        return 
    
#     print n[0].moves
#     raw_input('starting expand')
    
    if p.n_dice > 0:
#         raw_input('get moves')
        p.get_moves(n[0].moves)
#         raw_input(str(p.n_dice) + str(n[0].moves.size()) + str(p.dice))
        if n[0].moves.size()>0:
            n[0].role =<string>'max' if p.player == 0 else <string>'min'
        else:
            n[0].role = <string>'switch'
            n[0].moves.push_back(dummy_move) #dummy_move is basically passing the dice to the next player
    else:
#         raw_input('get chances')
        n[0].moves = ROLLS
        n[0].role = <string>'chance'
#     print n[0].moves
#     raw_input('get size')
    size = n[0].moves.size()
    
    assert(n[0].children.size() == 0)   
#     raw_input('setting leaf, N,V,Q')
    n[0].leaf = 0

    n[0].N.assign(size,0.)
    n[0].V.assign(size,0.)
    n[0].Q.assign(size,0.)
#     raw_input('setting children')
#     n[0].children.assign(size,n)
    
    for i in xrange(size):
        child = new Node()
        child[0].index = i  
        child[0].leaf = 1
        child[0].root = 0
        n[0].children.push_back(child)
        child[0].parent = n
        child[0].moves = test_moves

cdef string move_score(Node* n,int depth):
    cdef int i
    cdef string s
    s = ''
    for i in xrange(n[0].N.size()):
        s.append(<string>"{:2d}->{:2d} : N = {:6.0f}, Q = {:+1.4f} ".format(
            n[0].moves[i].start,n[0].moves[i].end, n[0].N[i], n[0].Q[i]))
        s.append(move_principal(n[0].children[i],depth))
        s.append(<string>'\n')
    return s

cdef string move_principal(Node* n, int depth):
    cdef string s
    cdef int i,index
    cdef double best
    s = ''

    if depth and (n[0].role==<string>'max' or n[0].role ==<string>'min'):
        s = ''
        index = -1
        best = -1
        for i in xrange(n[0].N.size()):
            if n[0].N[i] > best:
                index = i
                best = n[0].N[i]
        if index != -1:
            s.append(<string>"{:2d}->{:2d} ".format(n[0].moves[index].start,n[0].moves[index].end))
            s.append(move_principal(n[0].children[index] , depth - 1))
    return s

cdef string board_principal(Node* n,Position p,int depth):
    cdef int i,index
    cdef double best
    cdef string s
    s = ''

    if depth and (n[0].role==<string>'max' or n[0].role == <string>'min'):
        index = -1
        best = -1
        for i in xrange(n[0].N.size()):
            if n[0].N[i] > best:
                index = i
                best = n[0].N[i]
        if index != -1:
            p.play(n[0].moves[index])
            board_principal(n[0].children[index] , p , depth - 1)
        s = <string> p.print_board()
    return s

    
cdef int simulate(Position p):
    while not p.check_win():
        if p.n_dice:
            assert(p.dice[0] in range(1,7))
            p.play_random()
        else:
            p.roll()
    return p.check_win()

cdef int group_simulate(Position p):
    cdef vector[move] group
    cdef int i
    cdef vector[vector[move]] groups
    while not p.check_win():
        if p.n_dice:
            p.move_group(groups)
            if groups.size()==0:
                p.switch()
            else:
                group = groups[rng64()%groups.size()]
                for i in xrange(group.size()):
                    p.play(group[i])
                if p.n_dice:
                    p.switch()
        else:
            p.roll()
    return p.check_win()
            
            

cpdef p_simulate(Position p):
    return simulate(p)
cpdef p_group_simulate(Position p):
    return group_simulate(p)

from IPython.display import clear_output
cdef Node MC(Position p,duration = 5., int verbose = 1):
    cdef int log_len = p.ilog
    cdef Node* node
    cdef Node root
    cdef move m
    cdef int i,size
    cdef int index
    cdef int result
    cdef int moves_made = 0
    cdef string s
    cdef int principal_size
    cdef double best
    cdef int count
    cdef vector[move] moves
    
    
    count = 0    
    root.root = 1
    root.leaf = 1
#     p.get_moves(moves)
#     if moves.size() <= 1:
#         return root
    principal_size = p.n_dice
    
    t0 = time.clock()
    t60fps = time.clock()
    while time.clock() - t0 < duration:
        count += 1
#     for count in xrange(5000):
        #tree policy
        node = &root
        while not node[0].leaf:
            index = select(node) 
#             raw_input(str(index) + ' ' + str(node[0].moves.size()))
            if node[0].role == <string>'max' or node[0].role == <string>'min':
#                 raw_input('playing move' + str(p.dice))
                p.play(node[0].moves[index])
#                 raw_input('playing move' + str(p.dice))
            elif node[0].role == <string>'chance':
#                 raw_input('rolling')
                p.force_roll(node[0].moves[index].start,node[0].moves[index].end)
#                 raw_input('done rolling')
            elif node[0].role == <string>'switch':                
                p.switch()
            else:
                sys.exit('invalid role')
            node = node[0].children[index]
        #expand
        
        assert(node[0].leaf)
#         if node == &root:
#         raw_input('expand'+str(p.dice))
        expand(node,p)
        #simulate and backprop
#         raw_input('simulate and backprop'+str(p.dice))
        backprop(node,simulate(p))     
        while(p.ilog > log_len):
#             raw_input(str(p.ilog) + ' ' + str(log_len) + ' ' + str(p.dice))
            p.back()
#         raw_input('after back'+str(p.dice))
        if verbose and (time.clock() - t60fps > 1/24.):
#         if count%100==0:
            t60fps = time.clock()
            clear_output(wait = True)
            s = <string>('time elapsed: {:5.1f}s'.format(time.clock() - t0)  +'\n'    )
            s.append(board_principal(&root,p,principal_size))
            s.append(<string> '\n')
            s.append(move_score(&root,principal_size))
            print s
            sys.stdout.flush()
        while(p.ilog > log_len):
            p.back()

    if verbose:
        print depth(&root)
        print print_tree(&root,1)
    for child in root.children:
        del_tree(child)
    root.children.clear()
    if verbose:
        print depth(&root)
        print 'time elapsed: ' + str(time.clock() - t0)
        print 'runs: ' + str(count)
    return root
            
            
cpdef p_MC(Position p,float duration = 2, int verbose = 1):
    MC(p,duration,verbose)
    
cdef string print_node(Node n):
    return 'moves ' + str(n.moves) + '  N ' + str(n.N) + '  Q ' + str(n.Q)

cdef depth(Node* n):
    cdef int d,best
    best = -1
    for child in n[0].children:
        if child[0].leaf:
            d = 1
            if d>best:
                best = d
        else:            
            d = 1 + depth(child)
            if d > best:
                best = d
    return best

cdef del_node(Node*n):
    del n
cdef del_tree(Node*n):
    cdef Node* child
    for child in n[0].children:
        del_tree(child)
    del n
        
cpdef select_roll():
    r = rng64()%36
    i = -1
    while r >= 0:
        i += 1     
        r -= PROBS[i]
    return ROLLS[i]

######################### BOTS #########################
ctypedef move (*policy)(Position,double)
cdef move MC_bot_verbose(Position p , double duration): #insure there are moves to play before calling
    cdef Node root
    cdef int index,i
    cdef double best
    root = MC(p,duration,verbose = 1)    
    index = -1
    best = -1
    for i in xrange(root.moves.size()):
        if root.N[i] > best:
            best = root.N[i]
            index = i
    return root.moves[index]

cdef move MC_bot(Position p , double duration): #insure there are moves to play before calling
    cdef Node root
    cdef int index,i
    cdef double best
    root = MC(p,duration,verbose = 0)    
    index = -1
    best = -1
    for i in xrange(root.moves.size()):
        if root.N[i] > best:
            best = root.N[i]
            index = i
    return root.moves[index]

cdef move human(Position p, double duration):#duration can be ignored
    clear_output(wait = True)
    cdef int i
    cdef move m
    p.print_board()
    print 'dice: ' + ' '.join([p.dice[i] for i in xrange(p.n_dice)])  
    print 'available moves'
    for _ in p.p_get_moves():
        print _
    m.start = <int> int(raw_input('from'))
    m.end = <int> int(raw_input('to'))
    m.die = <int> int(raw_input('die'))
    return m

cdef move random(Position p, double _): #no need to time
    cdef vector[move] moves
    p.get_moves(moves)
    return moves[rng64() % moves.size()]

cdef int play_game(Position p,policy p1,policy p2,double duration1,double duration2,int verbose):
    cdef vector[move] moves
    while not p.check_win():
        if p.n_dice:
            assert(p.dice[0] in range(1,7))
            if verbose:
                clear_output(wait = True)
                print p.print_board()
                sys.stdout.flush()
            p.get_moves(moves)
            if moves.size():
                if p.player == 0:
                    p.play(p1(p,duration1))
                else:
                    p.play(p2(p,duration2))
            else:
                p.switch()
        else:
            p.roll()    
    return p.check_win()

def p_play_game(Position p,player1,player2,duration1 = 1,duration2 = 1,verbose = True):
    cdef policy p1,p2
    if player1 == 'human':
        p1 = human
    elif player1 == 'verbose':
        p1 = MC_bot_verbose
    elif player1 == 'quiet':
        p1 = MC_bot
    else:
        p1 = random
    if player2 == 'human':
        p2 = human
    elif player2 == 'verbose':
        p2 = MC_bot_verbose
    elif player2 == 'quiet':
        p2 = MC_bot
    else:
        p2 = random
    return play_game(p,p1,p2,duration1,duration2,verbose)

cdef string print_tree(Node* r,int from_bot = 1,int from_top = 0):
    cdef string s
    cdef int i
    s = <string> r[0].role
    if from_bot:
        s.append(<string> ('\n'+'\t'*from_top))
        for i in xrange(r[0].children.size()):
            s.append(<string>'\t')
            s.append(<string>"{:2d}->{:2d}".format(r[0].moves[i].start,r[0].moves[i].end))
            s.append(print_tree(r[0].children[i],from_bot-1,from_top+1))
            s.append(<string> ('\n'+'\t'*from_top))
    return s

    
            
    
    

In [44]:
t0 = time.clock()
score = 0
p = Position()
p.p_force_roll(2,4)
print p.p_get_move_group()

 6-> 2  6-> 4 
 6-> 2  8-> 6 
 6-> 2 13->11 
 6-> 2 24->22 
 6-> 4  6-> 2 
 6-> 4  8-> 4 
 6-> 4 13-> 9 
 6-> 4 24->20 
 8-> 4  4-> 2 
 8-> 4  6-> 4 
 8-> 4  8-> 6 
 8-> 4 13->11 
 8-> 4 24->22 
 8-> 6  8-> 4 
 8-> 6 13-> 9 
 8-> 6 24->20 
13-> 9  6-> 4 
13-> 9  8-> 6 
13-> 9  9-> 7 
13-> 9 13->11 
13-> 9 24->22 
13->11  6-> 2 
13->11  8-> 4 
13->11 13-> 9 
13->11 24->20 
24->20  6-> 4 
24->20  8-> 6 
24->20 13->11 
24->20 20->18 
24->20 24->22 
24->22  6-> 2 
24->22  8-> 4 
24->22 13-> 9 
24->22 24->20 

['6-2', '8-4', '13-9', '24-20', '6-4', '8-6', '13-11', '24-22']


In [9]:
p = Position()
p.p_force_roll(2,4)
p_MC(p,10)

time elapsed:  10.0s
12ooooo                     ooooo13
11                               14
10                               15
9                                16
8 oo                          ooo17
7                                18
6 ooooo                     ooooo19
5                                20
4 o                              21
3                               o22
2                                23
1 oo                            o24
0                                25






 6-> 2 : N =    631, Q = -0.0333 24->22 
 8-> 4 : N =   1161, Q = +0.0362 24->22 
13-> 9 : N =    642, Q = -0.0312 24->22 
24->20 : N =    908, Q = +0.0044 24->22 
 6-> 4 : N =   1080, Q = +0.0185  8-> 4 
 8-> 6 : N =    617, Q = -0.0373 24->20 
13->11 : N =    580, Q = -0.0448 13-> 9 
24->22 : N =    954, Q = +0.0094 13-> 9 

10
max
	 6-> 2max
	 8-> 4max
	13-> 9max
	24->20max
	 6-> 4max
	 8-> 6max
	13->11max
	24->22max

-1
time elapsed: 10.0273821751
runs: 6597


In [ ]:
p = Position()
p.p_set_board([0,0,0,2,0,0,0,0,3,0,0,-5,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
p.p_force_roll(3,3)
print p.print_board()
print p.p_get_move_group()

In [ ]:
d = {}
for i in xrange(100):
    p = Position()    
    result = p_play_game(p,'random','quiet',0.25,2,True)
    if result in d:
        d[result] += 1
    else:
        d[result] = 1
    
print d

In [ ]:
p = Position()
p.p_force_roll(3,3)
print p.print_vitals()
print p.print_board()
print p.p_get_move_group()

In [ ]:
for e in p.p_log():
    print e

In [ ]:
from IPython.display import clear_output
p= Position()
while not p.p_check_win():
    if p.p_get_n_dice():
        p.p_play_random()
    else:
        p.p_roll()
    p.print_board()
    p.print_vitals()
    print p.p_get_moves()
#     sys.stdout.flush()
    raw_input()
    clear_output()
   

In [ ]:
p = Position()
p.print_board()
p.print_vitals()
print p.p_get_moves()
p.p_force_roll(5,3)
p.print_board()
p.print_vitals()
print p.p_get_moves()


In [ ]:
from IPython.display import clear_output
# raw_input()
p= Position()
# raw_input()
p_simulate(p)
# raw_input()
p.print_board()
p.print_vitals()
print p.p_check_win()


# # raw_input()
while p.p_ilog():
#     p.print_board()
#     print len(p.p_log())
#     print p.p_ilog()
#     print p.p_log()[p.p_ilog()-1]?
#     print p.p_get_n_dice()
    sys.stdout.flush()
#     if len(p.p_log())%100 == 0:
#     p.print_board()
#     raw_input()
    p.p_back()
    
# p.p_roll()
p.print_board()
p.print_vitals()
# del p
# raw_input()
# p= Position()

# p_simulate(p)

In [ ]:
print p.p_ilog()

p.p_log()[:p.p_ilog()]

In [ ]:
import sys
import time
def play_random():  
    p = Position()
    return p_simulate(p)

score = {1:0,-1:0,-2:0,-3:0,2:0,3:0}
games = 10000
fps = 60.
t0 = time.clock()
t_flag = t0
prints=0
for i in xrange(games):    
    score[play_random()] += 1
    if time.clock() - t_flag > 1./fps:
        sys.stdout.write("\r" + ("{:4} "*6).format(*[score[-i - (i>=0)] for i in xrange(-3,3)]))
        sys.stdout.flush()
        t_flag = time.clock()
        prints+=1
print
print 'time',time.clock() - t0
print 'prints',prints
print 'pps' , prints/(time.clock() - t0)
print '''\nfirst player:
 backgammons - {: 2.1f}%
     gammons - {: 2.1f}%
regular wins - {: 2.1f}%

second player:
 backgammons - {: 2.1f}%
     gammons - {: 2.1f}%
regular wins - {: 2.1f}%
'''.format(*([score[i]*100./games for i in xrange(3,0,-1)]+[score[i]*100./games for i in xrange(-3,0)]))

print 'score: {:4} - {:4}'.format(3*score[3]+2*score[2]+score[1],3*score[-3]+2*score[-2]+score[-1])

In [ ]:
p = Position()
p.p_roll()
p.print_vitals()
print p.p_get_moves()
print p.p_get_n_dice()
while p.p_get_n_dice():
    p.p_play_random()
p.print_vitals()
p.print_board()
print p.p_get_moves()
print p.p_get_n_dice()

In [ ]:
p = Position()
p.print_vitals()
p.print_board()
p.p_roll()
p.print_vitals()
print(p.p_get_moves())
p.p_play_first()
p.print_vitals()
p.print_board()
print(p.p_get_moves())


In [ ]:
p = Position()
p.print_vitals()
p.print_board()
p.p_roll()
p.p_play(6,5)
p.print_vitals()
p.print_board()
p.p_switch()
p.p_play(1,5)
p.print_vitals()
p.print_board()
p.p_back()
p.print_vitals()
p.print_board()
p.p_switch()
p.p_back()
p.print_vitals()
p.print_board()

In [ ]:
%%cython -a --cplus


from libcpp.string cimport string

cdef string a
a = "Hello World"
print a

In [ ]:
%%cython -a --cplus
#cython: cdivision = True
from libc.stdlib cimport malloc,free
from libcpp.vector cimport vector
from libcpp.string cimport string

cdef struct vstruct:
    int a
    double b
    vector[int] v

cdef vstruct* vptr
vptr = <vstruct*> malloc(sizeof(vstruct))
vptr[0].a = 2
# vptr[0].b = 3.14159
# print vptr[0].a
vptr[0].v = [1,2,3,4]
print vptr[0].v
vptr[0].push_back(6)
print vptr[0].v
print vptr[0]


In [ ]:
%%cython -a --cplus
#cython: cdivision = True
from libc.stdlib cimport malloc,free
from libcpp.vector cimport vector
from libcpp.string cimport string

cdef struct vstruct:
    int a
    double b
    vector[int] v
    
cdef vstruct* vptr
vptr = <vstruct*> malloc(sizeof(vstruct))
vptr[0].a = 4
# vptr[0].b = 3.14159
vptr[0].v = [1,2]
print vptr[0].v
vptr[0].v.push_back(19)
print vptr[0].v
print vptr[0]

In [ ]:
%%cython -a --cplus
#cython: cdivision = True
from libc.stdlib cimport malloc,free
from libcpp.vector cimport vector
from libcpp.string cimport string

cdef struct vstruct:
    int a
    double b
    vector[int] v

    
cdef vstruct* vptr
vptr = <vstruct*> malloc(sizeof(vstruct))
vptr[0].a = 4
# vptr[0].b = 3.14159
vptr[0].v = [1,2]
print vptr[0].v
vptr[0].v.push_back(19)
print vptr[0].v
print vptr[0]

In [ ]:
%%cython -a

cdef extern from *:
    ctypedef int int128 "__int128_t"
    
# ctypedef unsigned long long int128
    
cdef int128 i,j
i = 0
# i = 1
# for j in xrange(1):    
#     print hex(i << j)